This code is intended to take the sketches and random labels in 1D and generate the image then generated image is feed to the 
Discriminator along with real images.

In [88]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import os
import wandb
import random
import torch.optim as optim
import torch.nn.functional as F

In [89]:
def random_label_generator():
    random_vector = [0.] * 7
    index = random.randint(0, 6)
    random_vector[index] = 1.
    return random_vector

In [90]:
class CustomDataLoader(Dataset):
    def __init__(self, image_folder, label_csv, sketch_folder, transform=None):
        self.transform = transform
        self.labels_df = pd.read_csv(label_csv)
        self.labels_df = self.labels_df.dropna()
        existing_images = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
        filtered_labels = []
        for idx, row in self.labels_df.iterrows():
            image=row['image']+'.jpg'
            if image in existing_images:
                filtered_labels.append(row)
        self.labels_df = pd.DataFrame(filtered_labels)
        
        self.image_folder = image_folder
        self.sketch_folder = sketch_folder
        self.sketch_transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
        ])
    
    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        # Load image
        image_name = self.labels_df.iloc[idx]['image'] + '.jpg'
        image_path = os.path.join(self.image_folder, image_name)
        image = Image.open(image_path).convert('RGB')
        
        # Load label
        label = self.labels_df.iloc[idx, 1:].values.astype(float)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        
        
        # Randomly sample sketch
        random_sketch_idx = random.randint(0, len(os.listdir(self.sketch_folder)) - 1)
        sketch_name = os.listdir(self.sketch_folder)[random_sketch_idx]
        sketch_path = os.path.join(self.sketch_folder, sketch_name)
        sketch = Image.open(sketch_path).convert('L')
        transformed_sketch = self.sketch_transform(sketch)
        
        # Generate random label image
        randomlabels = random_label_generator()
        random_label = torch.tensor(randomlabels, dtype=torch.float32)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label_tensor, transformed_sketch, random_label

In [91]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [92]:
pwd

'/workspace/ConditionalGAN'

In [93]:
# dataset and dataloader

train_sketch_folder='/workspace/ConditionalGAN/Dataset/Train/Train_contours'
train_image_folder='/workspace/ConditionalGAN/Dataset/Train/Train_data'
train_label_csv='/workspace/ConditionalGAN/Dataset/Train/Train_labels.csv'

test_sketch_folder='/workspace/ConditionalGAN/Dataset/Test/Test_contours'
test_image_folder='/workspace/ConditionalGAN/Dataset/Test/Test_data'
test_label_csv='/workspace/ConditionalGAN/Dataset/Test/Test_Labels.csv'


train_dataset = CustomDataLoader(train_image_folder,train_label_csv,train_sketch_folder,transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataLoader(test_image_folder,test_label_csv,test_sketch_folder,transform=transform)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [133]:
class Generator(nn.Module):
    def __init__(self, input_dim, label_dim, hidden_dim):
        super(Generator, self).__init__()

        self.input_dim = input_dim
        self.label_dim = label_dim
        self.hidden_dim = hidden_dim

        # Define the layers of the generator
        self.fc = nn.Linear(791, 512)
        self.conv_transpose1 = nn.ConvTranspose2d(hidden_dim, hidden_dim // 2, 4, 2, 1)
        self.batch_norm1 = nn.BatchNorm2d(hidden_dim // 2)
        self.conv_transpose2 = nn.ConvTranspose2d(hidden_dim // 2, hidden_dim // 4, 4, 2, 1)
        self.batch_norm2 = nn.BatchNorm2d(hidden_dim // 4)
        self.conv_transpose3 = nn.ConvTranspose2d(hidden_dim // 4, 3, 4, 2, 1)
        self.tanh = nn.Tanh()

    def forward(self, noise, label):
        # Concatenate noise and label
        #noise = noise.reshape(-1)
        #print(noise.shape, label.shape)
        size=noise.shape[0]
        self.hidden_dim=size*2
        #print(size)
        noise = noise.view(size, -1)
        #print(noise.shape)
        x = torch.cat((noise, label), dim=1)
        #print(x.shape)
        #x = x.permute(1,0)
        #print(x.shape)
        # Pass the concatenated tensor through the fully connected layer
        #print("Hi1")
        x = self.fc(x)
        #print(x.shape)
        #x = x.permute(0, 3, 1, 2)
        #x = x.permute(0, 2, 3, 1)
        #print(x.shape)
        #print("Hi2")
        x = x.contiguous()
        x = x.view(-1, self.hidden_dim, 16, 16)
        #print(x.shape)
        # Reshape the tensor for the convolutional layers
        #x = x.view(-1, self.hidden_dim, 4, 4)
        #x=x.reshape(512,self.hidden_dim, 128, 128)
        #print(x.shape)
        # Pass the tensor through the first transposed convolutional layer
        x = self.batch_norm1(self.conv_transpose1(x))
        x = self.tanh(x)
        #print(x.shape)
        # Pass the tensor through the second transposed convolutional layer
        x = self.batch_norm2(self.conv_transpose2(x))
        x = self.tanh(x)
        #print(x.shape)
        # Pass the tensor through the third transposed convolutional layer
        x = self.conv_transpose3(x)
        #print(x.shape)
        # Apply the hyperbolic tangent function to the output
        x = self.tanh(x)
        #print(x.shape)
        return x

In [101]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Discriminator, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        # Define the layers of the discriminator
        self.conv1 = nn.Conv2d(input_dim, hidden_dim, 4, 2, 1)
        self.batch_norm1 = nn.BatchNorm2d(hidden_dim)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim * 2, 4, 2, 1)
        self.batch_norm2 = nn.BatchNorm2d(hidden_dim * 2)
        self.conv3 = nn.Conv2d(hidden_dim * 2, hidden_dim * 4, 4, 2, 1)
        self.batch_norm3 = nn.BatchNorm2d(hidden_dim * 4)
        self.fc1 = nn.Linear(hidden_dim * 4 * 4, 1)

    def forward(self, image):
        # Concatenate image and label
        #label = label.permute(0, 3, 1, 2)
        #print(image.shape, label.shape)
        #x = torch.cat((image, label), dim=1)
        #x = torch.cat((image, random_label_image.unsqueeze(1).expand_as(image)), dim=1)
        #print(x.shape)
        
        x=image
        
        # Pass the concatenated tensor through the convolutional layers
        x = self.conv1(x)
        #print(x.shape)
        x = self.batch_norm1(x)
        x = self.leaky_relu(x)

        x = self.conv2(x)
        #print(x.shape)
        x = self.batch_norm2(x)
        x = self.leaky_relu(x)

        x = self.conv3(x)
        #print(x.shape)
        x = self.batch_norm3(x)
        x = x.view(-1, self.hidden_dim * 4 * 4)
        #print(x.shape)
        # Pass the flattened tensor through the fully connected layer
        x = self.fc1(x)
        x=F.sigmoid(x)
        #print(x.shape)
        return x

In [134]:
# Define the Generator and Discriminator
G = Generator(input_dim=1, label_dim=7, hidden_dim=64).cuda()
D = Discriminator(input_dim=3, hidden_dim=64).cuda()

# Define the loss function
criterion = nn.BCELoss()

# Define the optimizer for the generator and discriminator
optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [44]:
wandb.init(project="ConditionalGAN")

In [23]:
# wandb.config.update({
#     "input_dim": input_dim,
#     "label_dim": label_dim,
#     "hidden_dim": hidden_dim,
#     "batch_size": batch_size,
#     "num_epochs": num_epochs,
#     "lr": 0.0002,
#     "betas": (0.5, 0.999)
# })

In [97]:
num_epochs = 10
batch_size = 32

In [98]:
# # Train the cGAN
# for epoch in range(num_epochs):
#     for i, (images, labels, sketches, random_labels) in enumerate(train_loader):
#         #print(images.shape, labels.shape, sketches.shape, random_labels.shape)
#         #print("Train Descriminator")
#         # Train the discriminator
#         D.zero_grad()

#         # Train on real images
#         real_images = Variable(images.cuda())
#         real_labels = Variable(labels.cuda())
#         real_output = D(real_images, real_labels)
#         #print(real_output)
#         #real_loss = criterion(real_output, Variable(torch.ones(real_output.size(0)).cuda()))
#         real_loss = criterion(real_output, torch.ones_like(real_output))
#         real_loss.backward()

#         # Train on fake images
#         noise = Variable(sketches.cuda())
#         fake_labels = Variable(random_labels.cuda())
#         fake_images = G(noise, fake_labels)
#         fake_output = D(fake_images.detach(), fake_labels)
#         #fake_loss = criterion(fake_output, Variable(torch.zeros(fake_output.size(0)).cuda()))
#         fake_loss = criterion(fake_output, torch.ones_like(fake_output))
#         fake_loss.backward()

#         # Update the discriminator weights
#         D_loss = real_loss + fake_loss
#         optimizer_D.step()
        
#         #print("Train Generator")

#         # Train the generator
#         G.zero_grad()

#         # Train on fake images
#         noise = Variable(sketches.cuda())
#         fake_labels = Variable(random_labels.cuda())
#         fake_images = G(noise, fake_labels)
#         fake_output = D(fake_images, fake_labels)
#         #G_loss = criterion(fake_output, Variable(torch.ones(fake_output.size(0)).cuda()))
#         G_loss = criterion(fake_output, torch.ones_like(fake_output))
#         G_loss.backward()

#         # Update the generator weights
#         optimizer_G.step()

#     # Print the loss for each epoch
#     if epoch % 100 == 0:
#         print('Epoch [{}/{}], Step [{}/{}], D_loss: {:.4f}, G_loss: {:.4f}'
#               .format(epoch+1, num_epochs, i+1, len(train_loader), D_loss.item(), G_loss.item()))

In [135]:
for epoch in range(num_epochs):
    for i, (images, labels, sketches, random_labels) in enumerate(train_loader):
        # Train the discriminator
        print("Train the discriminator")
        D.zero_grad()

        # Train on real images
        real_images = Variable(images.cuda())
        real_labels = Variable(labels.cuda())
        real_output = D(real_images)
        real_loss = criterion(real_output, torch.ones_like(real_output))
        real_loss.backward()

        # Train on fake images
        noise = Variable(sketches.cuda())
        fake_labels = Variable(random_labels.cuda())
        fake_images = G(noise, fake_labels)
        fake_output = D(fake_images.detach())
        fake_loss = criterion(fake_output, torch.zeros_like(fake_output))
        fake_loss.backward()

        # Update the discriminator weights
        D_loss = real_loss + fake_loss
        optimizer_D.step()

        # Train the generator
        print("Train the generator")
        G.zero_grad()

        # Train on fake images
        noise = Variable(sketches.cuda())
        fake_labels = Variable(random_labels.cuda())
        fake_images = G(noise, fake_labels)
        fake_output = D(fake_images)
        G_loss = criterion(fake_output, torch.ones_like(fake_output))
        G_loss.backward()

        # Update the generator weights
        optimizer_G.step()

    # Log the training loss
    wandb.log({"epoch": epoch+1, "D_loss": D_loss.item(), "G_loss": G_loss.item()})

    # Perform validation/testing
    test_loss_D = 0.0
    test_loss_G = 0.0
    total_batches = 0

    for images, labels, sketches, random_labels in test_loader:
        with torch.no_grad():
            # Calculate discriminator loss on test set
            print("calculate the descriminator")
            real_images = images.cuda()
            real_labels = labels.cuda()
            real_output = D(real_images)
            real_loss = criterion(real_output, torch.ones_like(real_output))

            noise = sketches.cuda()
            fake_labels = random_labels.cuda()
            fake_images = G(noise, fake_labels)
            fake_output = D(fake_images)
            fake_loss = criterion(fake_output, torch.zeros_like(fake_output))

            test_loss_D += real_loss.item() + fake_loss.item()

            # Calculate generator loss on test set
            print("calculate the generator")
            noise = sketches.cuda()
            fake_labels = random_labels.cuda()
            fake_images = G(noise, fake_labels)
            fake_output = D(fake_images)
            G_loss = criterion(fake_output, torch.ones_like(fake_output))
            test_loss_G += G_loss.item()

            total_batches += 1

    # Average the test loss over all batches
    avg_test_loss_D = test_loss_D / total_batches
    avg_test_loss_G = test_loss_G / total_batches

    # Log the test loss
    wandb.log({"epoch": epoch+1, "test_D_loss": avg_test_loss_D, "test_G_loss": avg_test_loss_G})

    # Print the loss for each epoch
    print('Epoch [{}/{}], D_loss: {:.4f}, G_loss: {:.4f}, Test D_loss: {:.4f}, Test G_loss: {:.4f}'
          .format(epoch+1, num_epochs, D_loss.item(), G_loss.item(), avg_test_loss_D, avg_test_loss_G))

wandb.finish()

Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train the generator
Train the discriminator
Train th

RuntimeError: Given transposed=1, weight of size [64, 32, 4, 4], expected input[1, 62, 16, 16] to have 64 channels, but got 62 channels instead

In [ ]:
# Save the trained model
torch.save(G.state_dict(), 'generator.pth')
torch.save(D.state_dict(), 'discriminator.pth')

In [ ]:
# Load the trained model for inference
G_loaded = Generator(input_dim=1, label_dim=7, hidden_dim=64).cuda()
D_loaded = Discriminator(input_dim=3, label_dim=7, hidden_dim=64).cuda()

G_loaded.load_state_dict(torch.load('generator.pth'))
D_loaded.load_state_dict(torch.load('discriminator.pth'))

In [ ]:
# Generate and visualize some sample images using test loader
num_samples = 10
test_iter = iter(test_loader)
images, labels, sketches, random_labels = next(test_iter)

# Move data to GPU if available
images = images.cuda()
random_labels = random_labels.cuda()

# Generate images using the generator
generated_images = G(images, random_labels)

# Visualize the generated images
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(make_grid(generated_images.cpu().detach(), padding=2, normalize=True), (1, 2, 0)))
plt.show()